In [2]:
import sys
sys.path.append('/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone')
from src.dataset.dance_image import HumanDanceDataset
import matplotlib.pyplot as plt
import numpy as np
# import torch
import random
from PIL import Image
from src.utils.util import get_fps, read_frames, save_videos_from_pil
import cv2
from accelerate import Accelerator

from src.utils.util import (
    seed_everything,
)

from tqdm import tqdm
from insightface.app import FaceAnalysis

In [3]:
seed_everything(12580)

In [24]:
dataset = HumanDanceDataset(
    img_size=(512, 512),
    img_scale=(1.0, 1.0),
    img_ratio=(0.8, 1.0),
    data_meta_paths=["/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/data/hdtf_tiktok_meta.json"],
)

In [26]:
index = random.choice(np.arange(len(dataset)))
batch =  dataset.__getitem__(index)

In [34]:
raw_img = Image.fromarray((np.array(batch["raw_ref_img"].permute(1,2,0)*255)).astype(np.uint8))

In [33]:
app = FaceAnalysis(name='antelopev2', root='/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/pretrained_weights', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'cudnn_conv_use_max_workspace': '1', 'tunable_op_enable': '0'}}
find model: /cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/pretrained_weights/models/antelopev2/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'tunable_op_max_tuning_d

In [36]:
face_info = app.get(cv2.cvtColor(np.array(raw_img), cv2.COLOR_RGB2BGR))

/root/miniconda3/envs/animateAnyone/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [37]:
face_info = sorted(face_info, key=lambda x:(x['bbox'][2]-x['bbox'][0])*(x['bbox'][3]-x['bbox'][1]))[-1]

In [38]:
face_info

{'bbox': array([134.03757, 128.60904, 341.7697 , 419.44635], dtype=float32),
 'kps': array([[192.41939, 253.53955],
        [284.0232 , 232.90427],
        [253.7863 , 292.01373],
        [223.35129, 354.77258],
        [297.81494, 337.12   ]], dtype=float32),
 'det_score': 0.8563176,
 'landmark_3d_68': array([[ 1.2668950e+02,  2.6689297e+02,  1.4564366e+02],
        [ 1.3757976e+02,  2.9548825e+02,  1.4312686e+02],
        [ 1.4933046e+02,  3.2474857e+02,  1.3841405e+02],
        [ 1.6229170e+02,  3.5206741e+02,  1.2932016e+02],
        [ 1.7906688e+02,  3.7602808e+02,  1.0792444e+02],
        [ 1.9817961e+02,  3.9291254e+02,  8.8906883e+01],
        [ 2.1852251e+02,  4.0519217e+02,  7.2467751e+01],
        [ 2.4253632e+02,  4.1381744e+02,  5.1733944e+01],
        [ 2.7748203e+02,  4.1546454e+02,  4.4488033e+01],
        [ 3.0689630e+02,  3.9978711e+02,  5.7446556e+01],
        [ 3.2357074e+02,  3.7861844e+02,  8.3051041e+01],
        [ 3.3510886e+02,  3.5454150e+02,  1.0675129e+02],
